# Case Law Citations Extractor for EUR-LEX
Extracts case citations for all cases in EUR-LEX. This is done on the level-of-detail of the individual paragraph cited. For example, if we are extracting citations for case 62011CJ0488, then the citation 62010CJ0618: N 31 38 - 43 49 57 58 will be decomposed into the individual citations: 62010CJ0618: N31, 62010CJ0618: N38, 62010CJ0618: N39, 62010CJ0618: N40, 62010CJ0618: N41, 62010CJ0618: N42, 62010CJ0618: N43, 62010CJ0618: N49, 62010CJ0618: N57 and 62010CJ0618: N58

## Define main functions used in this notebook

### Functions: Part 1
Low-level functions for actually extracting metadata of each type for the given source case

In [484]:
# Urllib library used to query a website
from urllib.request import urlopen
# BeautifulSoup webscraping module for python
from bs4 import BeautifulSoup
# CSV parser
import csv
# Regular expressions
import re

#s = "123123STRINGabcabc"

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""


#print find_between( s, "123", "abc" ) -> 123STRING
#print find_between_r( s, "123", "abc" ) -> STRINGabc
        
def processProcedure(piece_of_text):
    #print(piece_of_text)
    # initialise list of items to be extracted from Procedure section
    items = []
    lines = piece_of_text.split('\n')
    lines2 = [x for x in lines if x]
    lines3 = []
    for thing in lines2:
        if (thing != '' and thing != ' '):
            lines3.append(thing)
            
    #print(lines3)
    
    judge = ''
    advocate = ''
    
    for item in lines3:
        
        line_split = item.split(': ')
        #print(line_split[0])
        if (((line_split[0].upper().count('JUDGE') > 0) or (line_split[0].upper().count('RAPPORTEUR') > 0)) and (judge == '')):
            judge = line_split[1]
        elif ((line_split[0].upper().count('ADVOCATE') > 0) and (advocate == '')):
            advocate = line_split[1]
            
    items.append(judge)
    items.append(advocate)
    
    #print(items)
    return items    
            
def processTitle(piece_of_text):
    # initialise list of items to be extracted from Title section
    items = []
    lines = piece_of_text.split('\n')
    lines2 = [x for x in lines if x]
    lines3 = []
    for thing in lines2:
        if (thing != '' and thing != ' '):
            lines3.append(thing)
            
    #print(lines3)
    
    line_split = lines3[0].split('.')
    line_split2 = [x for x in line_split if x]
    line_split3 = []
    for thing in line_split2:
        if (thing != '' and thing != ' '):
            line_split3.append(thing)
    
    #print(line_split3)
    
    ruling_title = line_split3[0]
    chamber = find_between_r(line_split3[0], '(', ')')
    ruling_name = line_split3[1]
    
    items.append(ruling_title)
    items.append(chamber)
    items.append(ruling_name)
    
    if (len(line_split3) == 5):
        for k in range(2, len(line_split3)-1):
            items.append(line_split3[k])
    else:
        items.append('Check EUR-LEX webpage')
        items.append('Check EUR-LEX webpage')
        
    case_label = line_split3[len(line_split3)-1]
    items.append(case_label)
    ecli = lines3[len(lines3)-1]
    items.append(ecli)
        
    #print(items)
    return items       
        
        
def processMisc(piece_of_text):
    # initialise list of items to be extracted from Miscellaneous section (Country)
    items = []
    lines = piece_of_text.split('\n')
    lines2 = [x for x in lines if x]
    lines3 = []
    for thing in lines2:
        if (thing != '' and thing != ' '):
            lines3.append(thing)
            
    country = ''
    
    for item in lines3:
        line_split = item.split(': ')
        if (line_split[0].upper().count('COUNTRY') > 0):
            country = line_split[1]
            
    items.append(country)
    
    #print(items)
    return items    
        
        
def processDates(piece_of_text):
    # initialise list of items to be extracted from Dates section (lodged and document dates)
    items = []
    lines = piece_of_text.split('\n')
    lines2 = [x for x in lines if x]
    dates = []
    for thing in lines2:
        if (thing != '' and thing != ' '):
            dates.append(thing)
            
    lodge_date = ''
    doc_date = ''
    
    for item in dates:
        date_split = item.split(': ')
        if (date_split[0].upper().count('LODGED') > 0):
            lodge_date = date_split[1]
        else:
            doc_date = date_split[1]
            
    items.append(lodge_date)
    items.append(doc_date)
    
    #print(items)
    return items

### Functions: Part 2
1) Low-level function for actually extracting the citations for a given source case, 2) function for extracting other subject matters related to a case, and 3) function to write data to file

In [485]:
# Urllib library used to query a website
from urllib.request import urlopen
# BeautifulSoup webscraping module for python
from bs4 import BeautifulSoup
# CSV parser
import csv
# Regular expressions
import re

# Write data (citations, metadata or subjects) to file
def writeToFile(subjectMatterCode, rows, datatype):
    with open('../data/judgements/'+datatype+'/'+subjectMatterCode+'_judgements_'+datatype+'.csv', 'a', newline='', encoding='utf-8') as csvfile:
        # Open file for writing
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        # Write each data row to file
        # Check if any element of list is also a list
        if (any(isinstance(el, list) for el in rows)):
            for row in rows:
                writer.writerow(row)
        else:
            writer.writerow(rows)

# Extract other subject matters for this source case (process the Classification section of the page)
def processClassifications(subjectMatterCode, piece_of_text, celexNumber):
    subjectMatters = find_between_r(piece_of_text, "Subject matter:", "Case law directory code:")
    splitStr = subjectMatters.split('\n')
    splitStr2 = [x for x in splitStr if x]
    subjectMattersFinal = []
    for thing in splitStr2:
        if (thing != '' and thing != ' '):
            subjectMattersFinal.append(thing)
    
    datarows = []
    for item in subjectMattersFinal:
        datarow = []
        datarow.append(celexNumber)
        tmp = item.replace(',', ';')
        tmp2 = tmp.replace('|', '')
        datarow.append(tmp2)
        datarows.append(datarow)
            
    writeToFile(subjectMatterCode, datarows, 'subjects')
        
# Filter out case citations that we are not interested in
def decomposeAndValidateCitation(citation, list_item):
    #print('citation: ' + str(citation))
    # Correct citations array
    correct_citations = []
    citation = citation.replace(" ", "")
    # Check if citation is a case (not a legislation or directive etc.)
    if (citation) and (citation[0] == '6' and citation[5] == 'C') and (citation[6] == 'J' or citation[6] == 'O'):
        # It is a valid citation, now decompose it
        # Part A: clean the last part of the citation e.g. the '-N10-14' part in '61980CJ0100-N10-14' -> '61980CJ0100: N10,
        # 61980CJ0100: N11, 61980CJ0100: N12, 61980CJ0100: N13, 61980CJ0100: N14'
        #print('----------------------------------------------------------')        
        #print() 
        #print('Citation: ' + str(citation))
        part_to_clean = citation[11:]
        if (part_to_clean.count(',') > 0):
            print('WTF')
            part_to_clean = part_to_clean.replace(',', ' ')
        #print('Part to clean: ' + str(part_to_clean))
        dash_count = part_to_clean.count('-')
        n_count = part_to_clean.count('N')
        if ((dash_count > 1) or (n_count > 1)):
            if (dash_count > 1):
                new_part_to_clean = part_to_clean[1:]
                #print('To Clean: ' + str(new_part_to_clean))
                guess = "-"
                occurrences = new_part_to_clean.count(guess)
                indices = [i for i, a in enumerate(new_part_to_clean) if a == guess]
                #print('Dashes: ' + str(indices))
                
                split_part = new_part_to_clean.split('-')
                lhs = split_part[0]
                rhs = split_part[1]
                number1 = re.sub("[^0-9]", "", lhs)
                number2 = re.sub("[^0-9]", "", rhs)
                for x in range(int(number1), int(number2)+1):
                    current_correct_citation = str(citation[:11]) + ': N' + str(x)
                    correct_citations.append(current_correct_citation)
            else:
                # It must be the case that there are two N's and a :
                # Remove trailing - and split on :
                new_part_to_clean = part_to_clean[1:].split(':')
                current_correct_citation = str(citation[:11]) + ': ' + str(new_part_to_clean[0])
                correct_citations.append(current_correct_citation)
                rhs = new_part_to_clean[1]
                rhs_numbers = rhs.split(' ')
                for rhs_number in rhs_numbers:
                    tmp = re.sub("[^0-9]", "", rhs_number)
                    if (len(tmp) != 0):
                        current_correct_citation = str(citation[:11]) + ': N' + str(rhs_number)
                        correct_citations.append(current_correct_citation)
        else:
            split_part = citation.split('-')
            #print('len split: ' + str(len(split_part)))
            if (len(split_part) > 1):
                correct_citation = str(split_part[0]) + ': ' + str(split_part[1]) 
                #print('Correct citation: ' + str(correct_citation))
                correct_citations.append(correct_citation)
            
        # Part B: add other paragraphs 
        # E.g. '61980CJ0100-N10-14: N 142' should include Part A decompositions + '61980CJ0100: N142'
        #print('another: ' + str(citation))
        test = list_item.text
        string_result = test.split('\n')
        for tmp in string_result:
            if (tmp.find(str(citation) + ':') != -1):
                #print('Part B: ' + str(tmp))
                split_again = tmp.split(':')
                stuff_to_clean = split_again[1]
                
                stuff_to_clean = stuff_to_clean.replace('-', ' - ')
                stuff_to_clean = re.sub(' +',' ',stuff_to_clean)
                #print('Stuff to clean: ' + str(stuff_to_clean))
                if (stuff_to_clean.count(',') > 0):
                    print('WTF2')
                    stuff_to_clean = stuff_to_clean.replace(',', ' ')
                #print('-------------------------')
                dash_count2 = 0
                dash_count2 = stuff_to_clean.count('-')
                if (dash_count2 > 0):
#                     print('To Clean: ' + str(stuff_to_clean))
#                     guess = "-"
#                     occurrences = stuff_to_clean.count(guess)
#                     indices = [i for i, a in enumerate(stuff_to_clean) if a == guess]
#                     print('Dashes: ' + str(indices))
                    
                     
                            
#                     print('After cleaning: ' + str(stuff_to_clean))        
                    
#                     split_string_dash = stuff_to_clean.split('-')
#                     lhs = split_string_dash[0]
#                     split_lhs = lhs.split(' ')
#                     split_lhs = list(filter(None, split_lhs))
#                     rhs = split_string_dash[1]
#                     split_rhs = rhs.split(' ')
#                     split_rhs = list(filter(None, split_rhs))
#                     last_num_lhs = split_lhs[len(split_lhs)-1]
#                     first_num_rhs = split_rhs[0]
            
#                     for x in range(int(last_num_lhs), int(first_num_rhs)+1):
#                         current_correct_citation = str(citation[:11]) + ': N' + str(x)
#                         correct_citations.append(current_correct_citation)
            
                    split_string_space = stuff_to_clean.split(' ')
                    for idx, val in enumerate(split_string_space):
                        if (val == '-'):
                            num1 = int(split_string_space[idx-1])
                            num2 = int(split_string_space[idx+1])
                            for x in range(num1, num2+1):
                                current_correct_citation = str(citation[:11]) + ': N' + str(x)
                                #print ("Current correct citation: " + str(current_correct_citation))
                                correct_citations.append(current_correct_citation)
                        else:
                            tmp = re.sub("[^0-9]", "", val)
                            if (len(tmp) != 0):
                                current_correct_citation = str(citation[:11]) + ': N' + str(val)
                                #print ("Current correct citation: " + str(current_correct_citation))
                                correct_citations.append(current_correct_citation)

                else:
                    split_string = stuff_to_clean.split(' ')
                    for num in split_string:
                        tmp = re.sub("[^0-9]", "", num)
                        if (len(tmp) != 0):
                            #print(num)
                            current_correct_citation = str(citation[:11]) + ': N' + str(num)
                            #print ("Correct citation: " + str(current_correct_citation))
                            correct_citations.append(current_correct_citation)
                    #print('-------------------------')
   
    return correct_citations

### Functions: Part 3
Main calling functions for extracting the citations and metadata for a given source case 

In [486]:
# Extract citations for the case given the BeautifulSoup format of it's HTML page
def extractCitations(subjectMatterCode, soup_judgement_page, celexNumber):
    # Citations array
    citations = []
    # Get all list items in this web page (the citations are in one of the list items in the HTML source)
    li_results = soup_judgement_page.find_all('li')
    # loop through items until you find the citation list item
    for result in li_results:
        # Check if this is the list item that lists all the citations for this judgement
        if (result.text[:11] == 'Instruments'):
            # If it is, loop through each, extract the citations, and write them to csv
            for link in result.find_all('a'):
                #if there is an href attribute
                if (link.has_attr('href')):
                    #extract the case number from the href
                    this_citation = link.text
                    #print("THIS CITATION:" + str(this_citation))
                    decomposedCitations = decomposeAndValidateCitation(this_citation, result)
                    # Remove empty items
                    decomposedCitations2 = list(filter(None, decomposedCitations))
                    for citation in decomposedCitations2:
                        if citation not in citations:
                            #print("Final correct citation: " + str(citation))
                            citations.append(citation)
                            
    # Initialise list of new data rows             
    datarows = []
    
    # For each citation
    for item in citations:
        # Initialise a new data row
        datarow = []
        # Split current citation into case code and paragraph number (e.g. 62008CJ0040: N 44 -> '62008CJ0040' + 'N44')
        split_citation = item.split(': ')
        # Add source case code to new data row
        datarow.append(celexNumber)
        # Add citation case code to new data row
        datarow.append(split_citation[0])
        # Add paragraph number to new data row
        datarow.append(split_citation[1])
        # Add new data row to the total data rows
        datarows.append(datarow)
     
    # Write total data rows to citations file
    writeToFile(subjectMatterCode, datarows, 'citations')
    
def findSectionType(result):
    results2 = result.find_all("div")
    for result2 in results2:
        if result2.get('class') is not None:
            if ((result2['class'][0]).count('boxTitle') == 1):
                return result2.text
    return ''

# Extract Metadata for case given the BeautifulSoup format of it's HTML page
def extractMetadata(subjectMatterCode, soup_judgement_page, celexNumber):
    # find all divs of class 'box'
    div_results = soup_judgement_page.find_all("div", {"class": "box"})
    # sections
    sections = []
    # datarow to write to file
    datarow = []
    datarow.append(celexNumber)
    # for each div of class 'box'
    for result in div_results:
        results2 = result.find_all("div")
        index = 0
        sectionType = findSectionType(result)
        sectionType = sectionType.replace(" ", "")
        #print("sectionType: " + str(sectionType))
        if (sectionType.count("Titleandreference") > 0) or (sectionType.count("Dates") > 0) or (sectionType.count("Procedure") > 0) or (sectionType.count("Classifications") > 0) or (sectionType.count("Miscellaneousinformation") > 0):
            #print("sectionType2: " + sectionType)
            # for each div inside 
            for result2 in results2:
                if result2.get('class') is not None:
                    if ((result2['class'][0]).count('tabContent') == 1):
                        if (sectionType.count("Titleandreference") > 0):
                            # 1. Title and reference
                            # Chamber, ruling name, ruling content, case label
                            #print(result2.text)
                            title = processTitle(result2.text)
                            datarow.extend(title)
                            
                        elif (sectionType.count("Dates") > 0):
                            # 2. Dates
                            # Date document, date lodged
                            #print(result2.text)
                            dates = processDates(result2.text)
                            datarow.extend(dates)
                            
                        elif (sectionType.count("Classifications") > 0):
                            #print(result2.text)
                            # 3. Classifications
                            # 3a. Subject matters
                            processClassifications(subjectMatterCode, result2.text, celexNumber)
                            
                        elif (sectionType.count("Miscellaneousinformation") > 0):
                            # 4. Misc
                            # Country
                            misc = processMisc(result2.text)
                            datarow.extend(misc)
                            
                        elif (sectionType.count("Procedure") > 0):
                            # 5. Procedure
                            # Judge-Rapporteur, Advocate General
                            procedure = processProcedure(result2.text)
                            datarow.extend(procedure)
    
    # Clean datarow items of all commas within each item
    cleaned_row = []
    for item in datarow:
        tmp = item.replace(',', ';')
        cleaned_row.append(tmp)
        
    cleaned_row.append(subjectMatterCode)
    #print(cleaned_row)
    #Write metadata row for source case to file
    writeToFile(subjectMatterCode, cleaned_row, 'metadata')


### Functions: Part 4
Functions for processing a single case and processing all cases within a main case subject

In [ ]:
from math import ceil
import json

# Process individual case given CELEX number
def processCase(subjectMatterCode, celexNumber):
    print("Source: " + str(celexNumber))
    #print('-------------------------')
    # URL prefix for a judgement on EUR-LEX
    result_url_prefix = "https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:"
    # URL for this particular judgement
    judgement_url = result_url_prefix + str(celexNumber)
    # Open the page
    judgement_page = urlopen(judgement_url)
    # Convert it to BeautifulSoup format 
    soup_judgement_page = BeautifulSoup(judgement_page, "lxml")
    # Get Metadata
    extractMetadata(subjectMatterCode, soup_judgement_page, celexNumber)
    # Get citations
    extractCitations(subjectMatterCode, soup_judgement_page, celexNumber)
    #print()

 # Process cases for a particular subject matter code
def processCases(subjectMatterCode):
    # URL prefix and suffix for judgements search results (url = prefix_url + subject matter code + suffix_url)
    prefix_url = "http://eur-lex.europa.eu/search.html?searchEq=true&qid=1524797649507&DB_TYPE_OF_ACT=judgment&CASE_LAW_SUMMARY=false&DTS_DOM=EU_LAW&CT_CODED="
    suffix_url = "&typeOfActStatus=JUDGMENT&type=advanced&lang=en&SUBDOM_INIT=EU_CASE_LAW&DTS_SUBDOM=EU_CASE_LAW"
    # Get the URL for all cases about this subject matter code
    url = prefix_url + subjectMatterCode + suffix_url
    # Open the URL
    url_page = urlopen(url)
    # Parse the HTML in the page, and store them in Beautiful Soup format using the 'lxml' parser
    soup_url_page = BeautifulSoup(url_page, "lxml")
    # The search results page displays 10 results at a time. 
    # Get the total number of 10-result pages for this subject matter.    
    j_onsubmit = soup_url_page.find('form', id='pagingForm').get('onsubmit')
    j_onsubmit = j_onsubmit.replace(" ", "")
    j_number = 1
    if (j_onsubmit):
        j_numberStr = j_onsubmit.split(",")[1] 
        j_numberStr = j_numberStr.replace(")", "");
        j_numberStr = j_numberStr.replace(" ", "");
        # Final number of result pages
        j_number = int(j_numberStr)
    else:
        j_scripts = soup_url_page.find_all('script', type='application/json')
        for script in j_scripts:
            print("script: " + script.text)
            script_text = json.loads(script.text)
            if 'search' in script_text:
                if (script_text['search']):
                    tmp = script_text['search']
                    num = tmp['count']
                    print("yay!: " + str(num))
                    if ((num/10) > 1):
                        j_number = ceil(num/10)
                
    
    print()
    print("total pages: " + str(j_number))
    print()
    # Get each result item tag (thats where the metadata for each case is found - including the CELEX number)
    j_results_on_first_page = soup_url_page.find_all('td', class_='leftMetadata')
            
    print()
    print("Page 1/" + str(j_number))
    print()
    # Process the 10 cases on the first page of the results
    for result in j_results_on_first_page:
        for ul in result.find_all('ul'):        
            for li in ul.find_all('li'):
                if (li.text[:13] == "CELEX number:"):
                    celex = li.text[14:]
                    if (celex[0] == '6' and celex[5] == 'C') and (celex[6] == 'J' or celex[6] == 'O'):
                        processCase(subjectMatterCode, celex)
    
    if (j_number > 1):
        # Process the other cases from Page 2 of results onwards
        for x in range(2, j_number+1):
            print()
            print("Page " + str(x) + "/" + str(j_number))
            print()
            # Get URL of Page x of results
            current_judgements_result_page_url = url + '&page=' + str(x) 
            # Open URL of Page x of results
            current_judgements_result_page = urlopen(current_judgements_result_page_url)
            # Store the HTML form of this page in BeautifulSoup format
            soup_current_judgements_page = BeautifulSoup(current_judgements_result_page, "lxml")
            # Get each result item tag (thats where the metadata for each case is found - including the CELEX number)
            results_on_page_x = soup_current_judgements_page.find_all('td', class_='leftMetadata')
            #Find the 10 cases on this page
            for result in results_on_page_x:
                for ul in result.find_all('ul'):
                    for li in ul.find_all('li'):
                        if (li.text[:13] == "CELEX number:"):
                            celex = li.text[14:]
                            if (celex[0] == '6' and celex[5] == 'C') and (celex[6] == 'J' or celex[6] == 'O'):
                                processCase(subjectMatterCode, celex)
    
    
    

### Main Procedure: Get the list of EUR-LEX subject matter codes from file and process the cases for each
Note: we have a list of all the EUR-LEX subject matters in "../data/SubjectMatterCodes.tsv" 

In [ ]:
# CSV parser
import csv

# Array storing EUR-LEX subject matter codes
subjectMatterCodes = []

# Transfer codes from TSV file to array
with open('../data/subject_matters_sorted.csv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter=',')
    for row in reader:
        subjectMatterCodes.append(row[0])


#processCase('FIN', '62000CJ0011')
# For each subject matter code in EUR-LEX
for subjectMatterCode in subjectMatterCodes:
    print(subjectMatterCode)
    processCases(subjectMatterCode)
    print()


FIN

total pages: 18


Page 1/18

Source: 62017CJ0042
Source: 62015CJ0593
Source: 62015CJ0599
Source: 62015CJ0436
Source: 62015CJ0584
Source: 62014CJ0260
Source: 62014CJ0059

Page 2/18

Source: 62014CJ0383
Source: 62013CJ0607
Source: 62014CJ0052
Source: 62014CJ0053
Source: 62013CJ0599
Source: 62013CJ0341
Source: 62012CJ0335
Source: 62011CJ0077

Page 3/18

Source: 62011CJ0314
Source: 62011CJ0670
Source: 62012CJ0370
Source: 62011CJ0669
Source: 62010CJ0113

Page 4/18

Source: 62010CJ0564
Source: 62010CJ0465
Source: 62010CJ0201
Source: 62009CJ0405

Page 5/18

Source: 62010CJ0023
Source: 62010CJ0131
Source: 62009CJ0367
Source: 62009CJ0132
Source: 62008CJ0334
Source: 62008CJ0442
Source: 62008CJ0423
Source: 62006CJ0038
Source: 62006CJ0239
Source: 62005CJ0461

Page 6/18

Source: 62005CJ0409
Source: 62005CJ0372
Source: 62005CJ0387
Source: 62005CJ0284
Source: 62005CJ0294
Source: 62008CJ0158
Source: 62007CJ0275
Source: 62007CJ0278
Source: 62007CJ0150

Page 7/18

Source: 62007CJ0281
Source: 62007C

Source: 61982CJ0235(01)

Page 9/25

Source: 61985CJ0081
Source: 61985CJ0025
Source: 61984CJ0204
Source: 61984CJ0067
Source: 61985CJ0042
Source: 61983CJ0183
Source: 61984CJ0239
Source: 61983CJ0211
Source: 61983CJ0214
Source: 61983CJ0172

Page 10/25

Source: 61984CJ0063
Source: 61984CJ0027
Source: 61984CJ0064
Source: 61984CJ0066
Source: 61984CJ0172
Source: 61983CJ0250
Source: 61983CJ0078
Source: 61984CJ0050
Source: 61983CJ0151
Source: 61983CJ0103

Page 11/25

Source: 61983CJ0209
Source: 61983CJ0081
Source: 61983CJ0222
Source: 61983CJ0036
Source: 61983CJ0224
Source: 61983CJ0083
Source: 61983CJ0009
Source: 61983CJ0008
Source: 61982CJ0348
Source: 61983CJ0010

Page 12/25

Source: 61982CJ0270
Source: 61982CJ0140
Source: 61983CJ0076
Source: 61983CJ0002
Source: 61982CJ0284
Source: 61982CJ0031
Source: 61982CJ0263
Source: 61982CJ0235
Source: 61982CJ0234
Source: 61982CJ0188

Page 13/25

Source: 61982CJ0179
Source: 61982CJ0265
Source: 61982CJ0317
Source: 61982CJ0168
Source: 61981CJ0303
Source: 6198

Source: 61998CJ0471
Source: 61998CJ0468
Source: 61998CJ0467

Page 23/27

Source: 61998CJ0472
Source: 62000CJ0255
Source: 62000CJ0360
Source: 61999CJ0255
Source: 62000CJ0005
Source: 62000CJ0055
Source: 61999CJ0480
Source: 61999CJ0221
Source: 61998CJ0377

Page 24/27

Source: 61999CJ0389
Source: 61999CJ0163
Source: 61998CJ0376
Source: 61999CJ0264
Source: 61998CJ0006
Source: 61997CJ0378
Source: 61996CJ0343
Source: 61996CJ0228
Source: 61997CJ0010

Page 25/27

Source: 61996CJ0231
Source: 61996CJ0279
Source: 61996CJ0260
Source: 61996CJ0266
Source: 61996CJ0160
Source: 61996CJ0129
Source: 61995CJ0094
Source: 61996CJ0052
Source: 61995CJ0299
Source: 61995CJ0058

Page 26/27

Source: 61995CJ0144
Source: 61993CJ0046
Source: 61994CJ0044
Source: 61994CJ0096
Source: 61993CJ0412
Source: 61990CJ0007
Source: 61989CJ0105
Source: 61988CJ0068
Source: 61986CJ0039
Source: 61986CJ0197

Page 27/27

Source: 61985CJ0059

PROT

total pages: 29


Page 1/29

Source: 62016CJ0016
Source: 62015CJ0434
Source: 62015CJ0598


Page 7/36

Source: 62015CJ0499
Source: 62014CJ0560
Source: 62014CJ0573
Source: 62015CJ0640
Source: 62015CJ0582
Source: 62015CJ0289
Source: 62015CJ0618
Source: 62015CJ0417
Source: 62016CJ0452
Source: 62016CJ0477

Page 8/36

Source: 62016CJ0453
Source: 62015CJ0212
Source: 62014CJ0465
Source: 62016CJ0439
Source: 62015CJ0428
Source: 62015CJ0195
Source: 62015CJ0429
Source: 62015CJ0135
Source: 62015CJ0294
Source: 62015CJ0185

Page 9/36

Source: 62014CJ0601
Source: 62014CJ0304
Source: 62015CJ0191
Source: 62016CJ0294
Source: 62015CJ0102
Source: 62015CJ0196
Source: 62015CJ0230
Source: 62015CJ0222
Source: 62015CJ0070
Source: 62014CJ0511

Page 10/36

Source: 62015CJ0012
Source: 62015CJ0155
Source: 62015CJ0063
Source: 62015CJ0047
Source: 62015CJ0241
Source: 62014CJ0559
Source: 62016CJ0108
Source: 62014CJ0558
Source: 62014CJ0572
Source: 62013CJ0366

Page 11/36

Source: 62014CJ0483
Source: 62015CJ0404
Source: 62015CJ0175
Source: 62015CJ0695
Source: 62014CJ0094
Source: 62014CJ0443
Source: 62014CJ029

Source: 62010CJ0419
Source: 62010CJ0467
Source: 62010CJ0210
Source: 62009CJ0028
Source: 62010CJ0224
Source: 62010CJ0083
Source: 62010CJ0120

Page 11/39

Source: 62009CJ0554
Source: 62010CJ0184
Source: 62010CJ0294
Source: 62009CJ0176
Source: 62010CJ0128
Source: 62009CJ0338
Source: 62009CJ0142
Source: 62008CJ0508

Page 12/39

Source: 62009CJ0063
Source: 62008CJ0533
Source: 62009CJ0124
Source: 62009CJ0122
Source: 62009CJ0018
Source: 62007CJ0402
Source: 62008CJ0301
Source: 62008CJ0527
Source: 62008CJ0464
Source: 62008CJ0557

Page 13/39

Source: 62008CJ0335
Source: 62008CJ0253
Source: 62008CJ0161
Source: 62007CJ0504
Source: 62006CJ0345
Source: 62008CJ0298
Source: 62007CJ0321
Source: 62007CJ0045
Source: 62006CJ0455
Source: 62007CJ0549

Page 14/39

Source: 62008CJ0018
Source: 62007CJ0001
Source: 62006CJ0305
Source: 62007CJ0173
Source: 62006CJ0334
Source: 62006CJ0329
Source: 62006CJ0308
Source: 62007CJ0417
Source: 62006CJ0096
Source: 62006CJ0037

Page 15/39

Source: 62005CJ0438
Source: 62006CJ

...

Source: 62005CJ0455
Source: 62005CJ0111
Source: 62006CJ0388
Source: 62005CJ0035
Source: 62005CJ0391
Source: 62005CJ0435
Source: 62005CJ0313
Source: 62006CJ0104
Source: 62005CJ0249
Source: 62005CJ0360

Page 57/117

Source: 62003CJ0197
Source: 62005CJ0401
Source: 62005CJ0090
Source: 62005CJ0240
Source: 62006CJ0013
Source: 62005CJ0005
Source: 62005CJ0290
Source: 62005CJ0140
Source: 62003CJ0475
Source: 62005CJ0345

Page 58/117

Source: 62004CJ0434
Source: 62005CJ0128
Source: 62004CJ0181
Source: 62005CJ0228
Source: 62005CJ0072
Source: 62004CJ0470
Source: 62005CJ0166
Source: 62004CJ0193
Source: 62004CJ0439
Source: 62005CJ0251

Page 59/117

Source: 62005CJ0106
Source: 62005CJ0089
Source: 62004CJ0393
Source: 62004CJ0264
Source: 62004CJ0494
Source: 62004CJ0517
Source: 62004CJ0430
Source: 62005CJ0098
Source: 62004CJ0509
Source: 62004CJ0384

Page 60/117

Source: 62004CJ0169
Source: 62004CJ0443
Source: 62004CJ0184
Source: 62004CJ0245
Source: 62004CJ0046
Source: 62004CJ0495
Source: 62004CJ0210
Sour

Source: 61991CJ0210
Source: 61991CJ0208
Source: 61991CJ0144

Page 95/117

Source: 61991CJ0114
Source: 61991CJ0105
Source: 61991CJ0163
Source: 61991CJ0074
Source: 61991CJ0262
Source: 61991CJ0050
Source: 61991CJ0049
Source: 61990CJ0343
Source: 61990CJ0163
Source: 61991CJ0131

Page 96/117

Source: 61991CJ0149
Source: 61991CJ0096
Source: 61990CJ0228
Source: 61991CJ0287
Source: 61990CJ0195
Source: 61990CJ0327
Source: 61990CJ0347
Source: 61991CJ0020
Source: 61990CJ0200
Source: 61990CJ0052

Page 97/117

Source: 61990CJ0078
Source: 61990CJ0164
Source: 61990CJ0100
Source: 61990CJ0035
Source: 61990CJ0202
Source: 61990CJ0097
Source: 61989CJ0153
Source: 61989CJ0152
Source: 61990CJ0060
Source: 61989CJ0287

Page 98/117

Source: 61989CJ0297
Source: 61989CJ0230
Source: 61990CJ0109
Source: 61989CJ0119
Source: 61988CJ0120
Source: 61989CJ0159
Source: 61989CJ0249
Source: 61989CJ0015
Source: 61988CJ0047
Source: 61989CJ0343

Page 99/117

Source: 61988CJ0367
Source: 61988CJ0208
Source: 61989CJ0186
Source: 61

Source: 62010CJ0559
Source: 62010CJ0443
Source: 62008CJ0403
Source: 62010CJ0138
Source: 62010CJ0215

Page 18/140

Source: 62010CJ0196
Source: 62010CJ0152
Source: 62010CJ0351
Source: 62010CJ0423
Source: 62009CJ0288
Source: 62010CJ0153
Source: 62009CJ0405
Source: 62009CJ0546
Source: 62009CJ0161
Source: 62010CJ0078

Page 19/140

Source: 62009CJ0494
Source: 62010CJ0011
Source: 62009CJ0488
Source: 62010CJ0012
Source: 62009CJ0273
Source: 62009CJ0137
Source: 62009CJ0339
Source: 62009CJ0421
Source: 62009CJ0108

Page 20/140

Source: 62009CJ0199
Source: 62009CJ0213
Source: 62009CJ0142
Source: 62009CJ0423
Source: 62009CJ0382
Source: 62009CJ0248
Source: 62009CJ0371
Source: 62009CJ0354
Source: 62009CJ0234
Source: 62008CJ0028

Page 21/140

Source: 62009CJ0075
Source: 62008CJ0370
Source: 62008CJ0230
Source: 62008CJ0446
Source: 62009CJ0123
Source: 62005CJ0433
Source: 62008CJ0386
Source: 62008CJ0373
Source: 62008CJ0333
Source: 62008CJ0264

Page 22/140

Source: 62008CJ0430
Source: 62008CJ0410
Source: 62


Page 59/140

Source: 61996CJ0292
Source: 61995CJ0265
Source: 61996CJ0143
Source: 61996CJ0325
Source: 61995CJ0382
Source: 61995CJ0337
Source: 61996CJ0201
Source: 61995CJ0189
Source: 61994CJ0159
Source: 61994CJ0157

Page 60/140

Source: 61994CJ0160
Source: 61995CJ0349
Source: 61995CJ0359
Source: 61996CJ0261
Source: 61995CJ0338
Source: 61995CJ0067
Source: 61996CJ0028
Source: 61995CJ0347
Source: 61996CJ0237
Source: 61995CJ0334

Page 61/140

Source: 61995CJ0130
Source: 61996CJ0142
Source: 61994CJ0090
Source: 61995CJ0316
Source: 61995CJ0034
Source: 61995CJ0242
Source: 61995CJ0114
Source: 61995CJ0097
Source: 61996CJ0114
Source: 61995CJ0368

Page 62/140

Source: 61996CJ0105
Source: 61995CJ0164
Source: 61995CJ0329
Source: 61995CJ0405
Source: 61995CJ0223
Source: 61994CJ0321
Source: 61995CJ0272
Source: 61995CJ0105
Source: 61995CJ0274
Source: 61995CJ0352

Page 63/140

Source: 61995CJ0358
Source: 61996CJ0103
Source: 61995CJ0192
Source: 61995CJ0047
Source: 61995CJ0038
Source: 61995CJ0267
Source: 61

Source: 61985CJ0402
Source: 61986CJ0045
Source: 61986CJ0058

Page 98/140

Source: 61984CJ0176
Source: 61984CJ0178
Source: 61986CJ0168
Source: 61986CJ0098
Source: 61985CJ0364
Source: 61985CJ0275
Source: 61984CJ0013
Source: 61985CJ0124
Source: 61985CJ0179
Source: 61985CJ0148

Page 99/140

Source: 61985CJ0080
Source: 61985CJ0355
Source: 61985CJ0385
Source: 61984CJ0095
Source: 61985CJ0222
Source: 61985CJ0203
Source: 61985CJ0183
Source: 61985CJ0050
Source: 61984CJ0103
Source: 61985CJ0087

Page 100/140

Source: 61985CJ0156
Source: 61984CJ0304
Source: 61985CJ0158
Source: 61985CJ0058
Source: 61985CJ0054
Source: 61985CJ0121
Source: 61985CJ0038
Source: 61985CJ0057
Source: 61984CJ0035
Source: 61984CJ0271

Page 101/140

Source: 61985CJ0065
Source: 61984CJ0188
Source: 61984CJ0121
Source: 61984CJ0214
Source: 61984CJ0192
Source: 61984CJ0247
Source: 61984CJ0290
Source: 61984CJ0182
Source: 61984CJ0227
Source: 61983CJ0281

Page 102/140

Source: 61984CJ0200
Source: 61984CJ0252
Source: 61984CJ0163
Source: